In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from sklearn.utils import resample
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [2]:
spark = SparkSession.builder.appName('logReg').getOrCreate()
df = spark.read.csv('/FileStore/tables/customer_churn.csv', header=True, inferSchema=True)

In [3]:
display(df)

Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
Cameron Williams,42.0,11066.8,0,7.22,8.0,2013-08-30T07:00:40.000+0000,"10265 Elizabeth Mission Barkerburgh, AK 89518",Harvey LLC,1
Kevin Mueller,41.0,11916.22,0,6.5,11.0,2013-08-13T00:38:46.000+0000,"6157 Frank Gardens Suite 019 Carloshaven, RI 17756",Wilson PLC,1
Eric Lozano,38.0,12884.75,0,6.67,12.0,2016-06-29T06:20:07.000+0000,"1331 Keith Court Alyssahaven, DE 90114","Miller, Johnson and Wallace",1
Phillip White,42.0,8010.76,0,6.71,10.0,2014-04-22T12:43:12.000+0000,"13120 Daniel Mount Angelabury, WY 30645-4695",Smith Inc,1
Cynthia Norton,37.0,9191.58,0,5.56,9.0,2016-01-19T15:31:15.000+0000,"765 Tricia Row Karenshire, MH 71730",Love-Jones,1
Jessica Williams,48.0,10356.02,0,5.12,8.0,2009-03-03T23:13:37.000+0000,"6187 Olson Mountains East Vincentborough, PR 74359",Kelly-Warren,1
Eric Butler,44.0,11331.58,1,5.23,11.0,2016-12-05T03:35:43.000+0000,"4846 Savannah Road West Justin, IA 87713-3460",Reynolds-Sheppard,1
Zachary Walsh,32.0,9885.12,1,6.92,9.0,2006-03-09T14:50:20.000+0000,"25271 Roy Expressway Suite 147 Brownport, FM 59852-6150",Singh-Cole,1
Ashlee Carr,43.0,14062.6,1,5.46,11.0,2011-09-29T05:47:23.000+0000,"3725 Caroline Stravenue South Christineview, MA 82059",Lopez PLC,1
Jennifer Lynch,40.0,8066.94,1,7.11,11.0,2006-03-28T15:42:45.000+0000,"363 Sandra Lodge Suite 144 South Ann, WI 51655-7561",Reed-Martinez,1


In [4]:
no_churn = df.filter(df['Churn']==0).count()
churn = df.filter(df['Churn']==1).count()
diff = (no_churn - churn) /2
upSampleLength = churn + diff
downSampleLength = no_churn - diff
downSampleLength

Out[42]: 450.0

In [5]:
df_no_churn = df.filter(df['Churn']==0)
df_no_churn.toPandas()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
0,Michael Williams,35.0,15571.26,0,6.45,9.0,2011-12-02 20:13:49,"5728 Michael Rue Riosland, NY 38804-1415",Petty and Sons,0
1,Connie Golden,39.0,10268.87,1,3.68,6.0,2009-08-19 18:52:21,"3324 Gomez Knolls Suite 591 Farmerchester, LA ...",Brown-Wagner,0
2,Seth Griffin,44.0,12328.03,1,4.60,9.0,2006-08-29 02:24:37,"9436 Warner Mill Suite 265 Port Kenneth, OH 00...",Williams PLC,0
3,Rachel Cherry,52.0,9782.83,0,3.96,7.0,2012-04-17 10:47:29,"3479 Stewart Way Sandersside, ID 69317-8759","Bishop, Tran and Pope",0
4,Robert Sanders,29.0,9378.24,0,4.93,8.0,2015-08-06 22:29:28,"8199 Christopher Tunnel Suite 537 Warnerside, ...","White, Jones and Nelson",0
5,Chris Sutton,37.0,10314.67,1,5.86,8.0,2016-06-22 19:20:04,"858 Booth Points South Davidside, AZ 92136-1145",Collins Ltd,0
6,Kenneth Alexander,30.0,8403.78,1,4.13,7.0,2006-06-28 09:26:57,"032 Jorge Trail Apt. 091 Brianmouth, ID 23964",Torres Inc,0
7,Jason Taylor,46.0,5570.45,0,2.23,7.0,2012-05-01 06:20:54,"693 Kevin Groves Apt. 158 Debramouth, NV 73809","Moore, Nichols and Miller",0
8,Dennis Morgan,43.0,8042.76,0,4.95,8.0,2006-04-29 20:25:59,"00103 Jeffrey Crest Apt. 205 Padillaville, IA ...","Barry, Jordan and Hall",0
9,Penny Perez,44.0,10309.15,1,6.35,9.0,2006-01-10 18:04:20,725 Stephanie Springs Apt. 350 South Brianbury...,Davis-Travis,0


In [6]:
#spparating churn and no_churn and converting to pandas dataframe
df_no_churn = df.filter(df['Churn']==0).toPandas()
df_churn = df.filter(df['Churn']==1).toPandas()
RANDOM_SEED = 10

#applying upsampling and downsampling
no_churn_downsampled = resample(df_no_churn,
              replace=True,
              n_samples=int(downSampleLength),
              random_state=10)
churn_upsampled = resample(df_churn,
              replace=True,
              n_samples=int(upSampleLength),
              random_state=10)

#concatenating pandas dataframe: churn and no_churn
pd_df = [no_churn_downsampled, churn_upsampled]
pd_df = pd.concat(pd_df)

#random shuffling
pd_df = pd_df.sample(frac=1).reset_index(drop=True)

#convert pandas df to spark df
df = spark.createDataFrame(pd_df)


In [7]:
display(df)

Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
James Griffin,52.0,9519.64,0,5.61,8.0,2015-12-05T19:35:08.000+0000,Unit 5498 Box 5539 DPO AP 33602,Walker Ltd,0
Jose Lee,45.0,12455.38,0,3.26,11.0,2015-02-24T14:54:33.000+0000,"036 Hunter Heights Suite 701 New Rickey, HI 95539-9622",Nelson LLC,1
Jonathan Brown,38.0,13019.89,1,6.65,8.0,2011-01-24T12:28:51.000+0000,"073 Ryan Haven Suite 323 New Jonathan, DC 25812-0853",Munoz-Pearson,0
Jason Jones MD,55.0,16838.94,1,6.66,8.0,2007-10-05T15:48:38.000+0000,"52454 Jessica Passage Apt. 097 Mitchellburgh, AS 30584-9004",Brown-Richardson,1
Kevin Mueller,41.0,11916.22,0,6.5,11.0,2013-08-13T00:38:46.000+0000,"6157 Frank Gardens Suite 019 Carloshaven, RI 17756",Wilson PLC,1
Mark Gordon,41.0,10028.23,1,5.52,9.0,2007-11-25T17:41:30.000+0000,"56171 Hunt Fall Herreratown, OR 59549",Cole-Johnson,0
Shannon Miller,35.0,7759.15,1,6.06,10.0,2013-07-18T22:25:07.000+0000,"826 Ross Causeway Wardshire, ME 25891",Nelson LLC,0
Kathryn Ponce,46.0,13725.55,0,5.09,9.0,2006-03-04T00:11:53.000+0000,"43798 Jones Garden Apt. 027 Monicashire, ND 82344","Cochran, Molina and Higgins",1
Matthew Clayton,41.0,8996.04,1,6.19,9.0,2011-04-20T06:33:42.000+0000,"866 Huffman Ports Suite 795 South Stacyport, KS 32492",Morgan and Sons,0
Maria Stanley,46.0,8941.92,1,5.43,9.0,2010-01-15T17:22:51.000+0000,"17836 Julie Drives North Thomas, MI 27198","Conner, Munoz and Wilson",0


In [8]:
display(df)

Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
James Griffin,52.0,9519.64,0,5.61,8.0,2015-12-05T19:35:08.000+0000,Unit 5498 Box 5539 DPO AP 33602,Walker Ltd,0
Jose Lee,45.0,12455.38,0,3.26,11.0,2015-02-24T14:54:33.000+0000,"036 Hunter Heights Suite 701 New Rickey, HI 95539-9622",Nelson LLC,1
Jonathan Brown,38.0,13019.89,1,6.65,8.0,2011-01-24T12:28:51.000+0000,"073 Ryan Haven Suite 323 New Jonathan, DC 25812-0853",Munoz-Pearson,0
Jason Jones MD,55.0,16838.94,1,6.66,8.0,2007-10-05T15:48:38.000+0000,"52454 Jessica Passage Apt. 097 Mitchellburgh, AS 30584-9004",Brown-Richardson,1
Kevin Mueller,41.0,11916.22,0,6.5,11.0,2013-08-13T00:38:46.000+0000,"6157 Frank Gardens Suite 019 Carloshaven, RI 17756",Wilson PLC,1
Mark Gordon,41.0,10028.23,1,5.52,9.0,2007-11-25T17:41:30.000+0000,"56171 Hunt Fall Herreratown, OR 59549",Cole-Johnson,0
Shannon Miller,35.0,7759.15,1,6.06,10.0,2013-07-18T22:25:07.000+0000,"826 Ross Causeway Wardshire, ME 25891",Nelson LLC,0
Kathryn Ponce,46.0,13725.55,0,5.09,9.0,2006-03-04T00:11:53.000+0000,"43798 Jones Garden Apt. 027 Monicashire, ND 82344","Cochran, Molina and Higgins",1
Matthew Clayton,41.0,8996.04,1,6.19,9.0,2011-04-20T06:33:42.000+0000,"866 Huffman Ports Suite 795 South Stacyport, KS 32492",Morgan and Sons,0
Maria Stanley,46.0,8941.92,1,5.43,9.0,2010-01-15T17:22:51.000+0000,"17836 Julie Drives North Thomas, MI 27198","Conner, Munoz and Wilson",0


In [9]:
df.show()

+-----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn|
+-----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 James Griffin|52.0| 9519.64| 0| 5.61| 8.0|2015-12-05 19:35:08|Unit 5498 Box 553...| Walker Ltd| 0|
 Jose Lee|45.0| 12455.38| 0| 3.26| 11.0|2015-02-24 14:54:33|036 Hunter Height...| Nelson LLC| 1|
 Jonathan Brown|38.0| 13019.89| 1| 6.65| 8.0|2011-01-24 12:28:51|073 Ryan Haven Su...| Munoz-Pearson| 0|
 Jason Jones MD|55.0| 16838.94| 1| 6.66| 8.0|2007-10-05 15:48:38|52454 Jessica Pas...| Brown-Richardson| 1|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|
 Mark Gordon|41.0| 10028.23| 1| 5.52| 9.0|2007-11-25 17:41:30|56171 Hunt Fall H...| Cole-Johnson| 0|
 Shannon Miller|35.0| 7759.15| 1| 6.06| 10.0|2013-07-18 22:25:07|826 Ross Causeway...| Nelson LLC| 0|
 Kathryn Ponce|46.0| 13725.55| 0| 5.09| 9.0|2006-03-04 00:11:53|43798 Jones Garde...|Cochran, Molina a...| 1|
 Matthew Clayton|41.0| 8996.04| 1| 6.19| 9.0|2011-04-20 06:33:42|866 Huffman Ports...| Morgan and Sons| 0|
 Maria Stanley|46.0| 8941.92| 1| 5.43| 9.0|2010-01-15 17:22:51|17836 Julie Drive...|Conner, Munoz and...| 0|
 Elizabeth Drake|40.0| 10930.16| 1| 6.63| 7.0|2014-05-28 16:08:41|540 Benjamin Vill...| Francis-Powers| 0|
 Tammy Shepherd|48.0| 11740.0| 1| 4.97| 11.0|2011-03-06 01:34:58|234 Gonzalez Terr...| Martin Ltd| 1|
 Abigail Jennings|36.0| 12309.23| 1| 4.76| 9.0|2012-03-11 17:10:26|USS Mcgrath FPO A...|Simpson, Byrd and...| 0|
 Mary Aguilar|50.0| 6244.75| 0| 4.64| 11.0|2015-12-18 13:28:17|Unit 6345 Box 013...| Lee-Aguilar| 1|
 Jason Santana|39.0| 9408.96| 1| 3.83| 4.0|2011-09-30 06:15:01|8072 Kathy Harbor...| Gonzalez LLC| 0|
Jennifer Carrillo|35.0| 12871.79| 0| 4.64| 12.0|2006-03-25 01:04:53|659 Robertson Lak...| Ingram-Swanson| 1|
 Brianna White|45.0| 9722.92| 1| 3.35| 8.0|2006-01-07 19:04:44|087 Kimberly Ridg...| Alexander LLC| 0|
 David Rodriguez|38.0| 8642.01| 1| 4.76| 11.0|2014-08-13 17:14:07|7318 Graham Ville...| Walker-Baker| 1|
 Russell Bauer|38.0| 7287.57| 1| 7.39| 11.0|2007-04-22 06:22:27|83221 Hendricks H...| Chambers-Simmons| 1|
 Casey Mcdaniel|41.0| 9918.77| 0| 4.02| 10.0|2016-03-04 04:04:37|Unit 5523 Box 760...| Bishop and Sons| 0|
+-----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [10]:
df.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [11]:
#selecting columns 
df_select_col = df.select(['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'Company', 'Churn'])

#drop na
df_select_col = df_select_col.na.drop()
df_select_col.printSchema()

root
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [12]:
#performing one hot encoding
company_indexer = StringIndexer(inputCol='Company', outputCol='CompanyIndex')
indexer = company_indexer.fit(df_select_col)
company_vec = indexer.transform(df_select_col)

In [13]:
# initialize assembler
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'CompanyIndex'], outputCol='features')
output = assembler.transform(company_vec)
# Check 
final_data = output.select('features', 'Churn')
display(final_data)

features,Churn
"List(1, 5, List(), List(52.0, 9519.64, 5.61, 8.0, 80.0))",0
"List(1, 5, List(), List(45.0, 12455.38, 3.26, 11.0, 0.0))",1
"List(1, 5, List(), List(38.0, 13019.89, 6.65, 8.0, 425.0))",0
"List(1, 5, List(), List(55.0, 16838.94, 6.66, 8.0, 29.0))",1
"List(1, 5, List(), List(41.0, 11916.22, 6.5, 11.0, 175.0))",1
"List(1, 5, List(), List(41.0, 10028.23, 5.52, 9.0, 178.0))",0
"List(1, 5, List(), List(35.0, 7759.15, 6.06, 10.0, 0.0))",0
"List(1, 5, List(), List(46.0, 13725.55, 5.09, 9.0, 39.0))",1
"List(1, 5, List(), List(41.0, 8996.04, 6.19, 9.0, 64.0))",0
"List(1, 5, List(), List(46.0, 8941.92, 5.43, 9.0, 182.0))",0


In [14]:
#train test split 
train_data, test_data = final_data.randomSplit([0.7,0.3], seed=10)


In [15]:
#define model
model = LogisticRegression(labelCol='Churn')
fitted_model = model.fit(train_data)
training_sum = fitted_model.summary
training_sum.predictions.describe().show()

+-------+-------------------+------------------+
summary| Churn| prediction|
+-------+-------------------+------------------+
 count| 631| 631|
 mean|0.49445324881141045|0.5007923930269413|
 stddev| 0.5003658762470161|0.5003960396660536|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+-------------------+------------------+

In [16]:
results = fitted_model.evaluate(test_data)
results.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|Churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[25.0,9672.03,5.4...| 0|[4.80335772983369...|[0.99186456799351...| 0.0|
[26.0,8787.39,5.4...| 1|[0.53559885750547...|[0.63078800718918...| 0.0|
[30.0,8677.28,7.3...| 0|[2.79029154491641...|[0.94214893722880...| 0.0|
[30.0,8874.83,5.5...| 0|[5.03677268074546...|[0.99354723369833...| 0.0|
[30.0,13473.35,3....| 0|[4.71129644226858...|[0.99108704422801...| 0.0|
[32.0,9472.72,1.0...| 0|[5.32671125674712...|[0.99516347848593...| 0.0|
[33.0,5738.82,7.6...| 0|[5.54058021721627...|[0.99609109027545...| 0.0|
[33.0,7750.54,4.5...| 0|[6.85052757684694...|[0.99894222318056...| 0.0|
[33.0,12638.51,5....| 0|[2.40517745474612...|[0.91722125923516...| 0.0|
[34.0,6131.92,4.1...| 0|[2.58449763670475...|[0.92985718531242...| 0.0|
[34.0,8772.26,5.7...| 0|[0.76466984381039...|[0.68236674766718...| 0.0|
[34.0,9401.99,8.2...| 0|[3.36603522565048...|[0.96662602328923...| 0.0|
[34.0,12983.26,6....| 1|[-4.7940206176228...|[0.00821112253488...| 1.0|
[34.0,12983.26,6....| 1|[-4.7940206176228...|[0.00821112253488...| 1.0|
[35.0,7361.92,4.9...| 0|[2.93858796344674...|[0.94972134385862...| 0.0|
[35.0,7759.15,6.0...| 0|[-2.4648228153798...|[0.07836132042635...| 1.0|
[35.0,8828.59,6.7...| 0|[3.22551096653971...|[0.96178309394353...| 0.0|
[35.0,9381.12,6.7...| 1|[-0.3178082563668...|[0.42120998523783...| 1.0|
[35.0,10801.37,5....| 0|[2.28354822966820...|[0.90750531417782...| 0.0|
[36.0,8233.25,4.3...| 0|[5.24962787590002...|[0.99477794155630...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [17]:
#evaluate
model_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')
auc = model_eval.evaluate(results.predictions)
print("auc: ", auc)

auc: 0.9213961721429361

In [18]:
%matplotlib inline
roc = training_sum.roc.toPandas()
plt.figure(figsize=(10,10))
plt.plot(roc['FPR'],roc['TPR'])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()
print('Training set areaUnderROC: ' + str(training_sum.areaUnderROC))
